In [3]:
# htmlからcsvに変換後に実行するツール。
# 全てのcsvを連結し、１年分にまとめる。
# その後に不要な言葉を整理する。
# 途中でミスっても良いように、処理は細切れにしています。

In [ ]:
#coding: UTF-8

# 出力されたCSVを連結する

import pandas as pd
import glob

# フォルダ内のcsvファイルをリスト形式で取得のはず
# ファイル名でソートする。初期のままだとソートされないため。
csv_files = sorted(glob.glob('*.csv'))

#読み込むファイルのリストを表示
for a in csv_files:
    print (a)

# csvファイルの中身を追加していくリストを準備
data_list = []

# 読み込むファイルのリストを走っていく
for file in csv_files:
    data_list.append(pd.read_csv(file))

# リストを全て行方向に連結
# axis=0:行方向に連結、ソートする
df =pd.concat(data_list, axis=0, sort=True)

# いったん保存する。
df.to_csv('All_議事録.csv', index=False)

In [250]:
# 保存したcsvをデータフレームに読み込む
df = pd.read_csv ('All_議事録.csv')

# csvを読み込んだデータフレームを「文字列」に変換する
df = df.applymap(str)

#"namae列"の先頭の数字と：を削除する
df["namae"] = df["namae"].replace("\d{1,5}.", "", regex=True).copy()

# 発言内容に発言者が入っているため、列名'voice_re'の名前を正規表現で全て削除する。
# 丸括弧の指定は要注意。最短で探すようにしないと、文の途中で不自然に切られてしまう。
df['voice_re'] = df['voice_re'].str.replace('^◯.+?\（.+?\）', '', regex=True).copy()

# 下記は臨時議会などで議長が決まっていないときは必要。
df['voice_re'] = df['voice_re'].str.replace('^◯.+?議員', '', regex=True).copy()

#発言した議員と、その内容が欲しい場合は、ここで終了。
#さらに処理を進める場合は、次に進む。

In [ ]:
# 議員と執行部の発言だけが欲しいときは、ココからの処理となる。
# 議長の発言の列に、進行上の記号や結果が多いので、不要と判断。
# ばっさりするので、使うときは要注意。
# 議長の名前は"直接"入れる。(なぜか正規表現か効かない？？ため)
df = df.loc[df['namae'] !='◯議長（郷内良治）']

# 議長の発言を全て削除の動作しない。正規表現が間違っているかも？
# 要改善のため残す
#df_clean2 = df.loc[df['namae'] !='^◯議長.+?\（.+?\）'].copy()

# 途中で確認してみる。
print(df)

In [252]:
# 臨時議長があるときは、ここでも削除の処理が必要。
# 臨時議長の正規表現が失敗するため、議事録を読んで直接指定する。
# 不要の場合は、飛ばしても可。
df = df.loc[df['namae'] !='◯臨時議長（大沼宗彦）']

In [253]:
# 
# 下記で出力するのは、議長を除いた発言のみ。(＝議員と執行部)
df.to_csv('合体_voice_re_only.csv', columns=['voice_re'], header=False, index=False)

In [254]:
# 発言の中で取り切れない記号があったので、ここで削除する。
# pandasでは上手くいかないため、csvファイルで加工する。
# ファイルの読み込み。
with open('合体_voice_re_only.csv') as reader:
    monyumonyu = reader.read()

# 置換というか削除
monyumonyu = monyumonyu.replace('─────────────────────○──────────────────────', '')

# ファイルに書き出し
with open('合体_voice_re_only_fixed.csv', 'w') as writer:
    writer.write(monyumonyu)


In [ ]:
# 下記は、試験的に失敗したモノ。次への構想として残しておく。
# 先頭に全角空白5文字文の行を削除する。
# 具体的には、「日程」「議案」などの区切りで、必ず全角空白５文字が頭に入っている。

#import re
# 読み込み元と書き出し先を同時にオープン
#with open('合体_voice_re_only.csv') as reader, open('合体_voice_re_fixed.csv', 'w') as writer:
    # 読み込み元の行ごとにイテレーション
#    for line in reader:
#        # 先頭に全角空白5文字文の行を削除する
#        line2 = re.subn('^　{5}.*\n', '', line)
        #line3 = re.subn('^s[5].*\n', '', line2)
        # 結果の書き出し
#        writer.write(line2)